# Dataset Index Generation
Generates indices for train, validation and test sets

In [2]:
import h5py
import re
import numpy as np

## Options

In [24]:
# define which files of events are included in test / validation / training split
# the same split is done for each particle type
test_files_start = 0
test_files_stop = test_files_start+10 # first 400 files are for test set
val_files_start = test_files_stop
val_files_stop = val_files_start+10 # next 100 files are for validation set
train_files_start = val_files_stop
train_files_stop = None # all remaining files are for training set
# train_files_stop = 500 # next 500 files are for training set

# define which particle labels to include
labels = (0, 1, 2, -1)

## Load dataset

In [25]:
data_path = "../data/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits_mini.h5"
f = h5py.File(data_path, "r")
event_labels = np.array(f['labels'])
root_files = np.array(f['root_files']).astype(str)

## Find the files of each label and indices of each file

In [26]:
def atoi(text):
    return int(text) if text.isdigit() else text
# Sort by only the basename of the file, with natural sorting of numbers in the filename
def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text.split('/')[-1]) ]

In [27]:
files_in_labels = {l: sorted(set(root_files[event_labels==l]), key=natural_keys) for l in labels}
idxs_in_files = {f: range(i, i+c) for f,i,c in zip(*np.unique(root_files, return_index=True, return_counts=True))}

In [28]:
for l, f in files_in_labels.items():
    print("label", l,"has", len(f),"files and ", sum([len(idxs_in_files[i]) for i in f]), "indices")

label 0 has 10 files and  30000 indices
label 1 has 10 files and  30000 indices
label 2 has 10 files and  30000 indices
label -1 has 10 files and  30000 indices


## Create the splits

In [43]:
split_files = {"test_idxs":  [f for l in labels for f in files_in_labels[l][0:10]],
               "val_idxs":   [f for l in labels for f in files_in_labels[l][10:20]],
               "train_idxs": [f for l in labels for f in files_in_labels[l][20:]]}
split_idxs = {k: [i for f in v for i in idxs_in_files[f]] for k, v in split_files.items()}

In [44]:
for s in split_files.keys():
    print(s,"has", len(split_files[s]),"files and", len(split_idxs[s]),"indices")

test_idxs has 40 files and 120000 indices
val_idxs has 0 files and 0 indices
train_idxs has 0 files and 0 indices


In [45]:
# Verify that all events are uniquely accounted for
all_indices = np.concatenate(list(split_idxs.values()))
print(len(event_labels))
print(len(all_indices))
print(len(set(all_indices)))

120000
120000
120000


## Save file

In [34]:
np.savez('IWCD_mPMT_Short_4_class_3M_emgp0_fixed_idxs.npz', **split_idxs)

In [35]:
indices = np.load('IWCD_mPMT_Short_4_class_3M_emgp0_fixed_idxs.npz')

In [36]:
indices

In [37]:
indices.files

['test_idxs', 'val_idxs', 'train_idxs']

In [40]:
indices['test_idxs']

array([     0,      1,      2, ..., 119997, 119998, 119999])